In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

pd.set_option('display.max_columns', 70)                       # 출력할 열의 최대개수
pd.set_option('display.max_colwidth', 80)                      # 출력할 열의 너비

In [66]:
# 필요한 데이터 불러오기
data1 = pd.read_csv('data_for_service_1.csv')
data2 = pd.read_excel('place-data-v.1.3.xlsx')
sim_order_table = pd.read_csv('유사도 순으로 장소 인덱스를 나열_10개만.csv')

In [3]:
# 여행 키워드 단어와 가중치를 입력하시오!
# 단 키워드 단어는 
# ['돈','쇼핑','시장','싸다, 저렴하다','비싸다','먹다','음식, 식사','상품','구매',
# '가격','호텔','박물관','전시관','관광지','유적지','미술관','역사','유익하다','문화','궁전','전쟁','활동','놀이공원','게임',
# '친구','커플','재미','아쿠아리움','엔터테인먼트','아이','가족','경기','힐링','공원','휴식','풍경','자연','전망대','산책',
# '분수','여유','나들이','공연','콘서트','이벤트','영화','예술','미술','강좌','요리','공예','체험'] 중에서 선택해야한다.

# 대략 5개의 키워드가 입력되었을 경우(실제로는 이렇게 안 되겠지만, 5개 상위 키워드로 뽑아서 진행할 것임)

keyword1 = input('첫 번째 여행 키워드는? : ')
weight1 = input('위 키워드의 가중치는? : ')
keyword2 = input('두 번째 여행 키워드는? : ')
weight2 = input('위 키워드의 가중치는? : ')
keyword3 = input('세 번째 여행 키워드는? : ')
weight3 = input('위 키워드의 가중치는? : ')
keyword4 = input('네 번째 여행 키워드는? : ')
weight4 = input('위 키워드의 가중치는? : ')
keyword5 = input('5 번째 여행 키워드는? : ')
weight5 = input('위 키워드의 가중치는? : ')

keyword = [keyword1, keyword2, keyword3, keyword4, keyword5]
weight = [int(weight1), int(weight2), int(weight3), int(weight4), int(weight5)]

user_data = pd.DataFrame({'keyword':keyword, 'weight':weight})
user_data['rate'] = user_data['weight']/(sum(user_data['weight']))
user_data = user_data.sort_values('rate', ascending = False)
user_data = user_data.reset_index()
print(user_data)

첫 번째 여행 키워드는? : 여유
위 키워드의 가중치는? : 10
두 번째 여행 키워드는? : 쇼핑
위 키워드의 가중치는? : 15
세 번째 여행 키워드는? : 휴식
위 키워드의 가중치는? : 10
네 번째 여행 키워드는? : 재미
위 키워드의 가중치는? : 5
5 번째 여행 키워드는? : 문화
위 키워드의 가중치는? : 10
   index keyword  weight  rate
0      1      쇼핑      15   0.3
1      0      여유      10   0.2
2      2      휴식      10   0.2
3      4      문화      10   0.2
4      3      재미       5   0.1


In [226]:
# 여행자 상의 두 타입 고르기
# def tour_type_1and2_with_weight(user_data):

tour_type_Table = {"tour_type" : ["Flex", "교양인", "놀래", "쉴래", "구경할래", "배울래"],
                   "sum_weight": [0,0,0,0,0,0]}
tour_type_Table = pd.DataFrame(tour_type_Table)

flex = 0
smart = 0
hangout = 0
rest = 0
sightseeing = 0
learning = 0

for i in range(len(user_data)):
    key_text = user_data.iloc[i,1]
    key_weight = user_data.iloc[i,3]

    if key_text in '돈, 쇼핑, 시장, 싸다, 저렴하다, 비싸다,먹다,음식, 식사, 상품,구매':
        flex +=key_weight
    if key_text in '박물관, 전시관,관광지,유적지,역사,유익하다,문화,궁전,전쟁':
        smart +=key_weight
    if key_text == '미술관':
        smart +=key_weight
    if key_text in '활동,놀이공원,게임,친구,커플,재미,아쿠아리움,엔터테인먼트,아이,가족,경기':
        hangout +=key_weight 
    if key_text in '힐링,공원,휴식,풍경,자연,전망대,산책,분수,여유,나들이':
        rest +=key_weight 
    if key_text in '공연,콘서트,이벤트,영화,예술,미술':
        sightseeing +=key_weight 
    if key_text in '강좌,요리,공예,체험':
        learning +=key_weight

tour_type_Table['sum_weight'] = [flex, smart, hangout, rest, sightseeing, learning]
tour_type_Table = tour_type_Table.sort_values('sum_weight', ascending = False)
tour_type_Table = tour_type_Table.reset_index()

first_user_tour_type = tour_type_Table.iloc[0,1]
first_user_tour_type_weight = float(tour_type_Table.iloc[0,2]) * 100
first_user_tour_type_num = tour_type_Table.iloc[0,0] + 1
second_user_tour_type = tour_type_Table.iloc[1,1]
second_user_tour_type_weight = float(tour_type_Table.iloc[1,2]) * 100
second_user_tour_type_num = tour_type_Table.iloc[1,0] + 1
thrid_user_tour_type_num = tour_type_Table.iloc[2,0] + 1

user_info = tour_type_1and2_with_weight(user_data)
user_info

['쉴래', 40.0, 4, 'Flex', 30.0, 1, 2]

In [227]:
tour_type_Table

,index,tour_type,sum_weight
0,3,쉴래,0.4
1,0,Flex,0.3
2,1,교양인,0.2
3,2,놀래,0.1
4,4,구경할래,0.0
5,5,배울래,0.0


In [229]:
# 가상 장소 확정 짓기 (키워드 별로 뽑기)
keyword_order = list(user_data['keyword'])
print(keyword_order)

# 키워드가 들어간 장소는 다 뽑기(쉬운 모델링을 위해 상위 3개만 고려함....)
firstword_table = data1[data1['tag_list'].str.contains(keyword_order[0])]
secondword_table = data1[data1['tag_list'].str.contains(keyword_order[1])]
thirdword_table = data1[data1['tag_list'].str.contains(keyword_order[2])]

entire_table = pd.concat([firstword_table,secondword_table,thirdword_table], axis=0)
entire_table = entire_table.drop_duplicates()
print('모으기')
print('entire_table 개수는 : ', len(entire_table))

if len(entire_table) >= 100:
    next_table = firstword_table
else : next_table = entire_table

print('next_table 개수는 : ', len(next_table))

# 한 번 걸러진,next_table에서 우선순위 단어 고려해보기
A1 = next_table[next_table['first_tag'] == keyword_order[0]]
A2 = next_table[next_table['second_tag'] == keyword_order[0]]
A3 = next_table[next_table['third_tag'] == keyword_order[0]]
important_A = pd.concat([A1,A2,A3])
important_A = important_A.drop_duplicates()

if len(next_table) >= 50:
    semi_table = A1
elif len(next_table) >= 20:
    semi_table = important_A
else : semi_table = next_table

print('semi_table 개수는 : ', len(semi_table))

B1 = semi_table[semi_table['first_tag'] == keyword_order[1]]
B2 = semi_table[semi_table['second_tag'] == keyword_order[1]]
B3 = semi_table[semi_table['third_tag'] == keyword_order[1]]
B4 = semi_table[semi_table['fourth_tag'] == keyword_order[1]]
B5 = semi_table[semi_table['fifth_tag'] == keyword_order[1]]

important_B = pd.concat([B1,B2,B3,B4,B5])
important_B = important_B.drop_duplicates()

if len(important_B)<= 5:
    final_table =semi_table
elif len(semi_table) >=30:
    if len(important_B[(important_B['first_tag'] == keyword_order[0]) & (important_B['second_tag'] == keyword_order[1])]) >= 1:
        final_table = important_B[(important_B['first_tag'] == keyword_order[0]) & (important_B['second_tag'] == keyword_order[1])]
    else : final_table = important_B[(important_B['first_tag'] == keyword_order[0]) | (important_B['second_tag'] == keyword_order[1])]
elif len(semi_table) >=20:
    final_table = important_B[important_B['first_tag'] == keyword_order[0]]
else : final_table = semi_table

print('final_table 개수는 : ', len(final_table))
    
# 최종 spot 선정
# '투어' 들어간 데이터 제외시키기
tour_table = final_table[final_table['place'].str.contains('투어')]
tour_table_index = tour_table.index.to_list()
final_table = final_table.drop(tour_table_index, axis=0)
print(final_table)

if len(final_table) >=2:
    end_table = final_table.sort_values('count', ascending=False).iloc[:3,:]
else : end_table = final_table

# 최종 spot 장소( 3개 이하)
print(end_table)
print(len(end_table))

['쇼핑', '여유', '휴식', '문화', '재미']
모으기
entire_table 개수는 :  400
next_table 개수는 :  241
semi_table 개수는 :  90
final_table 개수는 :  90
        id                place  \
5        6                 고투 몰   
6        7           신세계 백화점 본점   
7        8                타임스퀘어   
9       10  라인프렌즈 플래그십 스토어 명동역점   
10      11            스타필드 코엑스몰   
...    ...                  ...   
834   1662          충무로 애완동물 거리   
858   1730                 한복소녀   
872   1746         도령아씨 한복대여전문점   
1112  2260               코엑스 센터   
1123  2297               디보 빌리지   

                                                      tag_list  count  \
5         ['쇼핑', '가격', '싸다, 저렴하다', '상품', '시장', '구매', '음식, 식사']     33   
6        ['쇼핑', '음식, 식사', '비싸다', '먹다', '상품', '시장', '싸다, 저렴하다']     92   
7        ['쇼핑', '음식, 식사', '영화', '비싸다', '시장', '먹다', '싸다, 저렴하다']    103   
9            ['쇼핑', '상품', '아이', '친구', '구매', '싸다, 저렴하다', '비싸다']     38   
10    ['쇼핑', '음식, 식사', '비싸다', '호텔', '싸다, 저렴하다', '아쿠아리움', '먹다']    227   
...              

In [230]:
# end_table이 완성이 되었다면, 여기서 1~3 개 정도의 spot을 가상으로 잡아, 유사도 높은 장소를 각가 2개씩 뽑아오기
# 원래는 이 과정에서 여행 타입 별로 (비율적으로 분할해야한다...)
virtual_place_name = []
virtual_place_id = []
virtual_place_index = end_table.index.to_list()

for i in range(len(end_table)):
    vname = end_table.iloc[i,1]
    vid = end_table.iloc[i,0]
    virtual_place_name.append(vname)
    virtual_place_id.append(vid)
print(virtual_place_name)
print(virtual_place_id)
print(virtual_place_index)

['명동 쇼핑 거리', '봉은사', '이화여자대학교']
[1430, 1423, 1427]
[624, 617, 621]


In [231]:
# 각 장소 별 2군데 추가로 유사도로 선정

recommend_id = []
recommend_name = []
recommend_index = []
recommend_tourtype = []

for i in range(3):
    recom = sim_order_table[sim_order_table['0']== virtual_place_name[i]]
    spot1 = recom.iloc[0,1]
    spot2 = recom.iloc[0,2]
    spot3 = recom.iloc[0,3]
    spot4 = recom.iloc[0,4]
    spot5 = recom.iloc[0,5]
    spot6 = recom.iloc[0,6]
    spot7 = recom.iloc[0,7]
    spot8 = recom.iloc[0,8]
    
    recommend_index.append(virtual_place_index[i])
    recommend_index.append(spot1)
    recommend_index.append(spot2)
    recommend_index.append(spot3)
    recommend_index.append(spot4)
    recommend_index.append(spot5)
    recommend_index.append(spot6)
    recommend_index.append(spot7)
    recommend_index.append(spot8)
    
for j in recommend_index:
    find_name = sim_order_table.iloc[j,0]
    recommend_name.append(find_name)

for k in recommend_index:
    find_id = data1.iloc[k,0]
    recommend_id.append(find_id)

for t in recommend_id:
    find_type = list(data2[data2['id']== t]['tour_type'])[0]
    recommend_tourtype.append(find_type)
    
print(recommend_index)
print(recommend_name)
print(recommend_id)
print(recommend_tourtype)

[624, 30, 643, 136, 393, 29, 621, 996, 155, 617, 634, 1121, 628, 117, 832, 558, 509, 493, 621, 624, 524, 643, 41, 15, 42, 393, 29]
['명동 쇼핑 거리', '압구정 로데오 거리', '이대 패션거리', '문정동 로데오거리', '메가박스 코엑스몰점', '커먼그라운드', '이화여자대학교', '커피프린스 1호점', '미플 2001아울렛불광점', '봉은사', '종묘', '롯데월드 스타 애비뉴', '삼청동길', '동대문 헌책방 거리', '황학정국궁전시관', '굿모닝 약손마사지', '돈암문화스포츠센터', '스페셜박스 코인노래연습장', '이화여자대학교', '명동 쇼핑 거리', '실로암 불가마 사우나', '이대 패션거리', '두타', '롯데백화점 본점', '망원시장', '메가박스 코엑스몰점', '커먼그라운드']
[1430, 31, 1449, 143, 833, 30, 1427, 2001, 170, 1423, 1440, 2294, 1434, 122, 1660, 1160, 962, 942, 1427, 1430, 1122, 1449, 42, 16, 43, 833, 30]
[4, 1, 4, 1, 3, 1, 4, 1, 1, 4, 4, 3, 4, 1, 5, 1, 1, 3, 4, 4, 4, 4, 1, 1, 1, 3, 1]


In [127]:
# 최종 추천지 반환하기!!! (중복 당연히 제거)

final_recommend_id = []
final_recommend_name = []
final_recommend_tourtype = []
jungbok = []
jungsang = []

# 리뷰수가 10개 이상은 되는 곳으로!!
for a in recommend_id:
    if data1[data1['id'] == a].iloc[0,3]>=10:
        final_recommend_id.append(a)

# 중복제거를 위의 순서를 고려해서 하기!!(무작위 set안됨.)
for b in final_recommend_id:
    if b in jungsang:
        jungbok.append(b)
    else : jungsang.append(b)

for k in jungsang:
    final_name = data1[data1['id']==k].iloc[0,1]
    final_recommend_name.append(final_name)
    
for k in jungsang:
    final_tourtype = list(data2[data2['id']==k]['tour_type'])[0]
    final_recommend_tourtype.append(final_tourtype)

    
print("총 추천 결과는??") 
Recommendation_total = pd.DataFrame({'id':jungsang, 'name' : final_recommend_name, 'tour_type': final_recommend_tourtype})
Recommendation_total

총 추천 결과는??


,id,name,tour_type
0,1430,명동 쇼핑 거리,4
1,31,압구정 로데오 거리,1
2,1449,이대 패션거리,4
3,143,문정동 로데오거리,1
4,833,메가박스 코엑스몰점,3
5,30,커먼그라운드,1
6,1427,이화여자대학교,4
7,2001,커피프린스 1호점,1
8,1423,봉은사,4
9,1440,종묘,4


In [234]:
# 상위 두 타입에 맞게 소팅하기

# 상위 2개가 아닌 여행자 타입 구하기
for_extra = list(set(Recommendation_total['tour_type'].to_list()))
main = [user_info[2], user_info[5]]
for_extra = [x for x in for_extra if x not in main]

table1 = Recommendation_total[Recommendation_total['tour_type']==user_info[2]]
table2 = Recommendation_total[Recommendation_total['tour_type']==user_info[5]]
table3_extra = Recommendation_total[Recommendation_total['tour_type']==for_extra[0]]

if len(table1)<=4:
    plus_index = 4-len(table1)
    plus = table3_extra.iloc[:int(plus_index)+1,:]
    final_table1 = pd.concat([table1,plus])
else : final_table1 = table1.iloc[:5,:]
    
if len(table2)<=4:
    plus_index = 4-len(table2)
    plus = table3_extra.iloc[:int(plus_index)+1,:]
    final_table2 = pd.concat([table2,plus])
else : final_table2 = table2.iloc[:5,:]

In [235]:
final_table1

,id,name,tour_type
0,1430,명동 쇼핑 거리,4
2,1449,이대 패션거리,4
6,1427,이화여자대학교,4
8,1423,봉은사,4
9,1440,종묘,4


In [236]:
final_table2

,id,name,tour_type
1,31,압구정 로데오 거리,1
3,143,문정동 로데오거리,1
5,30,커먼그라운드,1
7,2001,커피프린스 1호점,1
13,42,두타,1
